In [4]:
# pandas for handling data
import pandas as pd
# google bigquery library for quering data
from google.cloud import bigquery
# BigQueryHelper for converting query result direct to dataframe
import bq_helper
from bq_helper import BigQueryHelper
# matplotlib for plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# import plotly
import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as fig_fact
plotly.tools.set_config_file(world_readable=True, sharing='public')

%matplotlib inline
# for loading data to mongodb
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\Shekhar\Downloads\TrainingUA2018Shekhar-55c13c58255f.json"

In [5]:
patents = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="epa_historical_air_quality")

# View table names under the epa_historical_air_quality data table
bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
bq_assistant.list_tables()



['air_quality_annual_summary',
 'co_daily_summary',
 'co_hourly_summary',
 'hap_daily_summary',
 'hap_hourly_summary',
 'lead_daily_summary',
 'no2_daily_summary',
 'no2_hourly_summary',
 'nonoxnoy_daily_summary',
 'nonoxnoy_hourly_summary',
 'o3_daily_summary',
 'o3_hourly_summary',
 'pm10_daily_summary',
 'pm10_hourly_summary',
 'pm25_frm_daily_summary',
 'pm25_frm_hourly_summary',
 'pm25_nonfrm_daily_summary',
 'pm25_nonfrm_hourly_summary',
 'pm25_speciation_daily_summary',
 'pm25_speciation_hourly_summary',
 'pressure_daily_summary',
 'pressure_hourly_summary',
 'rh_and_dp_daily_summary',
 'rh_and_dp_hourly_summary',
 'so2_daily_summary',
 'so2_hourly_summary',
 'temperature_daily_summary',
 'temperature_hourly_summary',
 'voc_daily_summary',
 'voc_hourly_summary',
 'wind_daily_summary',
 'wind_hourly_summary']

In [6]:
# View information on all columns in the co_daily_summary data table
bq_assistant.table_schema("co_daily_summary")



,name,type,mode,description
0,state_code,STRING,NULLABLE,The FIPS code of the state in which the monito...
1,county_code,STRING,NULLABLE,The FIPS code of the county in which the monit...
2,site_num,STRING,NULLABLE,A unique number within the county identifying ...
3,parameter_code,INTEGER,NULLABLE,The AQS code corresponding to the parameter me...
4,poc,INTEGER,NULLABLE,This is the “Parameter Occurrence Code” used t...
5,latitude,FLOAT,NULLABLE,The monitoring site’s angular distance north o...
6,longitude,FLOAT,NULLABLE,The monitoring site’s angular distance east of...
7,datum,STRING,NULLABLE,The Datum associated with the Latitude and Lon...
8,parameter_name,STRING,NULLABLE,The name or description assigned in AQS to the...
9,sample_duration,STRING,NULLABLE,The length of time that air passes through the...


In [7]:
QUERY = """
    SELECT
        co_daily.state_name,
        avg(co_daily.aqi) as co_avg_aqi,
        avg(no_daily.aqi) as no_avg_aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.co_daily_summary` AS co_daily
    INNER JOIN `bigquery-public-data.epa_historical_air_quality.no2_daily_summary` AS no_daily
        ON co_daily.state_name = no_daily.state_name
    WHERE
      co_daily.poc = 1
      AND no_daily.poc = 1
      AND EXTRACT(YEAR FROM co_daily.date_local) = 2017
      AND EXTRACT(YEAR FROM no_daily.date_local) = 2017
    GROUP BY co_daily.state_name
        """

bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")
df_states_gas = bq_assistant.query_to_pandas(QUERY)

In [8]:
df_states_gas.head()

,state_name,co_avg_aqi,no_avg_aqi
0,Oklahoma,4.028633,16.003973
1,Rhode Island,5.684062,16.401078
2,New York,2.109527,22.824577
3,Tennessee,3.725738,20.526027
4,Wyoming,1.892903,7.298307


In [9]:
QUERY = """
    SELECT
        so2_daily.state_name,
        avg(so2_daily.aqi) as so2_avg_aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.so2_daily_summary` AS so2_daily
    WHERE
      so2_daily.poc = 1
      AND EXTRACT(YEAR FROM so2_daily.date_local) = 2017
    GROUP BY so2_daily.state_name
        """
df_states_gas_so2 = bq_assistant.query_to_pandas(QUERY)
df_states_gas_so2.head()

,state_name,so2_avg_aqi
0,Nebraska,5.128915
1,Nevada,1.289875
2,Vermont,0.200000
3,Rhode Island,0.327397
4,Mississippi,0.596750


In [10]:
QUERY = """
    SELECT
        o3_daily.state_name,
        avg(o3_daily.aqi) as o3_avg_aqi
    FROM
      `bigquery-public-data.epa_historical_air_quality.o3_daily_summary` AS o3_daily
    WHERE
      o3_daily.poc = 1
      AND EXTRACT(YEAR FROM o3_daily.date_local) = 2017
    GROUP BY o3_daily.state_name
        """
df_states_gas_o3 = bq_assistant.query_to_pandas(QUERY)
df_states_gas_o3.head()

,state_name,o3_avg_aqi
0,Montana,37.153819
1,Alaska,26.782881
2,Iowa,36.601811
3,Ohio,40.216337
4,Texas,36.833493


In [11]:
df_states_gas['o3_avg_aqi'] = df_states_gas['state_name'].map(df_states_gas_o3.set_index('state_name')['o3_avg_aqi'])
df_states_gas['so2_avg_aqi'] = df_states_gas['state_name'].map(df_states_gas_so2.set_index('state_name')['so2_avg_aqi'])

In [12]:
df_states_gas.head()

,state_name,co_avg_aqi,no_avg_aqi,o3_avg_aqi,so2_avg_aqi
0,Oklahoma,4.028633,16.003973,39.244044,4.143517
1,Rhode Island,5.684062,16.401078,39.142125,0.327397
2,New York,2.109527,22.824577,34.903411,2.245774
3,Tennessee,3.725738,20.526027,40.261366,11.788858
4,Wyoming,1.892903,7.298307,43.374163,4.167829


In [13]:
#Insert into MongoDb
#Declare the database
db = client.AQI_db

In [15]:
data = df_states_gas.to_dict(orient='records')
db.states_aqi.insert_many(data)

In [16]:
# Verify results:
results = db.states_aqi.find()
for result in results:
    print(result)

{'_id': ObjectId('5bf21989e123ae4b4cbe0db4'), 'state_name': 'Oklahoma', 'co_avg_aqi': 4.028632784538203, 'no_avg_aqi': 16.003972758227835, 'o3_avg_aqi': 39.244044290347865, 'so2_avg_aqi': 4.143517092700956}
{'_id': ObjectId('5bf21989e123ae4b4cbe0db5'), 'state_name': 'Rhode Island', 'co_avg_aqi': 5.684062059238391, 'no_avg_aqi': 16.40107775211731, 'o3_avg_aqi': 39.14212548015365, 'so2_avg_aqi': 0.32739726027397265}
{'_id': ObjectId('5bf21989e123ae4b4cbe0db6'), 'state_name': 'New York', 'co_avg_aqi': 2.109526525955382, 'no_avg_aqi': 22.824577025823054, 'o3_avg_aqi': 34.90341134470445, 'so2_avg_aqi': 2.245773524720893}
{'_id': ObjectId('5bf21989e123ae4b4cbe0db7'), 'state_name': 'Tennessee', 'co_avg_aqi': 3.7257383966244495, 'no_avg_aqi': 20.526027397260552, 'o3_avg_aqi': 40.26136551496114, 'so2_avg_aqi': 11.788857938718664}
{'_id': ObjectId('5bf21989e123ae4b4cbe0db8'), 'state_name': 'Wyoming', 'co_avg_aqi': 1.8929032258064533, 'no_avg_aqi': 7.2983072916664975, 'o3_avg_aqi': 43.37416297517

In [52]:
states = {'AL': 'Alabama',
'AK': 'Alaska',
'AZ':'Arizona',
'AR':'Arkansas',
'CA':'California',
'CO':'Colorado',
'CT':'Connecticut',
'DE':'Delaware',
'FL':'Florida',
'GA':'Georgia',
'HI':'Hawaii',
'ID':'Idaho',
'IL':'Illinois',
'IN':'Indiana',
'IA':'Iowa',
'KS':'Kansas',
'KY':'Kentucky',
'LA':'Louisiana',
'ME':'Maine',
'MD':'Maryland',
'MA':'Massachusetts',
'MI':'Michigan',
'MN':'Minnesota',
'MS':'Mississippi',
'MO':'Missouri',
'MT':'Montana',
'NE':'Nebraska',
'NV':'Nevada',
'NH':'New Hampshire',
'NJ':'New Jersey',
'NM':'New Mexico',
'NY':'New York',
'NC':'North Carolina',
'ND':'North Dakota',
'OH':'Ohio',
'OK':'Oklahoma',
'OR':'Oregon',
'PA':'Pennsylvania',
'RI':'Rhode Island',
'SC':'South Carolina',
'SD':'South Dakota',
'TN':'Tennessee',
'TX':'Texas',
'UT':'Utah',
'VT':'Vermont',
'VA':'Virginia',
'WA':'Washington',
'WV':'West Virginia',
'WI':'Wisconsin',
'WY':'Wyoming'}

In [53]:
df_states = pd.DataFrame.from_dict(states,orient='index').reset_index()
df_states.columns = ['code', 'code_name']
df_states_gas['state_code'] = df_states_gas['state_name'].map(df_states.set_index('code_name')['code'])

In [54]:
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]


data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_states_gas['state_code'],
        z = df_states_gas['o3_avg_aqi'].astype(float),
        locationmode = 'USA-states',
        text =  'Average AQI of NO: ' + df_states_gas['no_avg_aqi'].astype(str) + '<br>' + 'Average AQI of CO: ' + df_states_gas['co_avg_aqi'].astype(str) + '<br>' + 'Average AQI of SO2: ' + df_states_gas['so2_avg_aqi'].astype(str),
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "AQI of O3")
        ) ]

layout = dict(
        title = 'The average air quality index of some dangerous element in different US states<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )




    
fig = dict( data=data, layout=layout )

py.iplot( fig, filename='d3-cloropleth-map' )
